# Automated Body Definition for RELION multibody refinement

this notebook describes possible automated approaches to define bodies readily usable in RELION 

## A. Load useful python libraries
We will package our tools in a library soon, in the meantime, make sure you have `our_library` point to the proper directory:

In [1]:
our_library = '/home/fpoitevi/notebooks/cryo_home/slaclab/cryoEM-notebooks/src/'

In [2]:
# just run this cell, no need to edit
%matplotlib inline
import shutil, os, sys, glob
import numpy as np
from matplotlib import pyplot as plt
import ipyvolume as ipv
sys.path.append(our_library)
import cryoemio
import imutils
import dataviz
import mrcutils
import clusterutils

## B. Define inputs and outputs

It is convenient to store the path to our project directory into a variable

In [3]:
root_directory = '/home/fpoitevi/notebooks/cryo_home/slaclab/cryoEM-notebooks/notebooks/Material/Heterogeneity/Multibody/'

**INPUTS** - In our case, we store the input data in the root directory. The input map is the output from a local-resolution job in RELION.

In [4]:
data_dir  = root_directory
input_map = data_dir+'/relion_locres_filtered.mrc'

**OUTPUTS** - The output directory is at the same location, and we define a common `keyword` for all our output files

In [5]:
body_dir = root_directory+'output/'
keyword  = 'bodies_'
output_mask  = body_dir+'/'+keyword # no need to edit this line

## C. Load and visualize the input map

We store in `data` the map stored in file `input_map`

In [6]:
# just run this cell, no need to edit
data = mrcutils.mrc2data(input_map)

In the following, we will work on thresholded maps, that is we will set to 0 the value of the voxels that have a value below `std_level` the standard deviation over the whole map. So first we define `std_level`

In [7]:
std_level = 1
std_map = np.std(data) # no need to edit this line

we now proceed to threshold the map and store the resulting map in `data_thresh` that we can visuzalize

In [8]:
# just run this cell, no need to edit
data_thresh = mrcutils.mrc_select(input_map, mode='above_value', value=std_level*std_map)
ipv.quickvolshow(data_thresh, level=[std_level*std_map, 3*std_level*std_map], opacity=0.03, level_width=std_level*std_map, data_min=np.min(data_thresh), data_max=np.max(data_thresh))

/opt/rh/rh-python36/root/usr/lib/python3.6/site-packages/ipyvolume/serialize.py:81: RuntimeWarning: invalid value encountered in true_divide
  gradient = gradient / np.sqrt(gradient[0]**2 + gradient[1]**2 + gradient[2]**2)


## D. Define the bodies

The idea is to build a `mask_dry` mask that is a highly blurred binarized version of a highly thresholded map, and a `mask_fat` mask that is a softly blurred binarized version of a softly thresholded map. The main body is the intersection of the two, and the remaining bodies are segmented from what remains.

$B_0 = M_{fat} \cap M_{dry}$ and $\bigcup_{i>1}^{K} B_{i} = M_{fat} \setminus B_0 $

In [9]:
# just run this cell, no need to edit
body0, bodyK = mrcutils.cut_thresholded_map(input_map, low_threshold=1*std_map, high_threshold=6*std_map)
mrcutils.data2mrc(output_mask+'body_0.mrc',body0,mrc_template=input_map)
mrcutils.data2mrc(output_mask+'body_K.mrc',bodyK,mrc_template=input_map)

At that stage, we have written out the mask of the main bodies and the union of the rest, that we can now segment out. 
We can control the rough size of the bodies by defining a typical length scale `length_scale` (in pixel units)

In [17]:
length_scale = 10

In [18]:
# just run this cell, no need to edit
labelled_map = clusterutils.segment_map(bodyK, length_scale=length_scale)

We can get a rough idea of the resulting body decomposition by sliding the `levels` slider below. If too many or not enough levels appear, change `length_scale` and re-run the cell above.

In [19]:
ipv.quickvolshow(labelled_map, level=[0, float(np.max(labelled_map))], opacity=0.03)

Once a reasonable segmentation has been achieved, we can write one map per body. They will be written in order of decreasing volume.

In [20]:
mrcutils.data2mrc(output_mask+'body_K_watershed_seg.mrc', labelled_map, mrc_template=input_map)
mrcutils.seg2mask(output_mask+'body_K_watershed_seg.mrc', output_mask+'body_K_watershed', sigma_blur=1,sort='volume',verbose=True)

/home/fpoitevi/notebooks/cryo_home/slaclab/cryoEM-notebooks/notebooks/Material/Heterogeneity/Multibody/output//bodies_body_K_watershed1.mrc > volume = 60090.0
/home/fpoitevi/notebooks/cryo_home/slaclab/cryoEM-notebooks/notebooks/Material/Heterogeneity/Multibody/output//bodies_body_K_watershed2.mrc > volume = 54080.0
/home/fpoitevi/notebooks/cryo_home/slaclab/cryoEM-notebooks/notebooks/Material/Heterogeneity/Multibody/output//bodies_body_K_watershed3.mrc > volume = 50832.0
/home/fpoitevi/notebooks/cryo_home/slaclab/cryoEM-notebooks/notebooks/Material/Heterogeneity/Multibody/output//bodies_body_K_watershed4.mrc > volume = 39704.0
/home/fpoitevi/notebooks/cryo_home/slaclab/cryoEM-notebooks/notebooks/Material/Heterogeneity/Multibody/output//bodies_body_K_watershed5.mrc > volume = 29656.0
/home/fpoitevi/notebooks/cryo_home/slaclab/cryoEM-notebooks/notebooks/Material/Heterogeneity/Multibody/output//bodies_body_K_watershed6.mrc > volume = 21258.0
/home/fpoitevi/notebooks/cryo_home/slaclab/cry

At that point, a visual inspection (in Chimera) leads to following decision:
- body 0: 0,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23
- body 1: 1
- body 2: 2
- body 3: 3
- body 4: 4
- body 5: 5
- body 6: 6

We merge the files now

In [40]:
# final body 0 
shutil.copyfile(output_mask+'body_0.mrc', 'tmp.mrc')
for i in [7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23]:
    mrcutils.mrc_algebra('tmp.mrc',output_mask+'body_K_watershed'+str(i)+'.mrc','tmp2.mrc')
    os.remove('tmp.mrc')
    os.rename('tmp2.mrc', 'tmp.mrc')
mrcutils.mrc2mask('tmp.mrc', output_mask+'body_0_final.mrc', sigma_blur=0., threshold=0.1)
os.remove('tmp.mrc')
# final bodies 1 to 6
for i in np.arange(1,7):
    shutil.copyfile(output_mask+'body_K_watershed'+str(i)+'.mrc', output_mask+'body_'+str(i)+'_final.mrc')

The files corresponding to the maps for the 6 bodies generated are listed below:

In [44]:
glob.glob(body_dir+'*final*')

['/home/fpoitevi/notebooks/cryo_home/slaclab/cryoEM-notebooks/notebooks/Material/Heterogeneity/Multibody/output/bodies_body_2_final.mrc',
 '/home/fpoitevi/notebooks/cryo_home/slaclab/cryoEM-notebooks/notebooks/Material/Heterogeneity/Multibody/output/bodies_body_0_final.mrc',
 '/home/fpoitevi/notebooks/cryo_home/slaclab/cryoEM-notebooks/notebooks/Material/Heterogeneity/Multibody/output/bodies_body_4_final.mrc',
 '/home/fpoitevi/notebooks/cryo_home/slaclab/cryoEM-notebooks/notebooks/Material/Heterogeneity/Multibody/output/bodies_body_5_final.mrc',
 '/home/fpoitevi/notebooks/cryo_home/slaclab/cryoEM-notebooks/notebooks/Material/Heterogeneity/Multibody/output/bodies_body_1_final.mrc',
 '/home/fpoitevi/notebooks/cryo_home/slaclab/cryoEM-notebooks/notebooks/Material/Heterogeneity/Multibody/output/bodies_body_3_final.mrc',
 '/home/fpoitevi/notebooks/cryo_home/slaclab/cryoEM-notebooks/notebooks/Material/Heterogeneity/Multibody/output/bodies_body_6_final.mrc']

They are ready to be imported in RELION, and the only remaining step is manual editing of a `bodies.star` also input to RELION to run the multibody refinement job, see https://elifesciences.org/articles/36861